<a href="https://colab.research.google.com/github/starryesh22/Google_Colab/blob/main/0608_QRSDetectorOffline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


from biosppy import tools
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter
import wfdb

from time import gmtime, strftime

LOG_DIR = "logs/"
PLOT_DIR = "plots/"


class QRSDetectorOffline(object):
    """
    Python Offline ECG QRS Detector based on the Pan-Tomkins algorithm.

    Michał Sznajder (Jagiellonian University) - technical contact (msznajder@gmail.com)
    Marta Łukowska (Jagiellonian University)


    The module is offline Python implementation of QRS complex detection in the ECG signal based
    on the Pan-Tomkins algorithm: Pan J, Tompkins W.J., A real-time QRS detection algorithm,
    IEEE Transactions on Biomedical Engineering, Vol. BME-32, No. 3, March 1985, pp. 230-236.

    The QRS complex corresponds to the depolarization of the right and left ventricles of the human heart. It is the most visually obvious part of the ECG signal. QRS complex detection is essential for time-domain ECG signal analyses, namely heart rate variability. It makes it possible to compute inter-beat interval (RR interval) values that correspond to the time between two consecutive R peaks. Thus, a QRS complex detector is an ECG-based heart contraction detector.

    Offline version detects QRS complexes in a pre-recorded ECG signal dataset (e.g. stored in .csv format).

    This implementation of a QRS Complex Detector is by no means a certified medical tool and should not be used in health monitoring. It was created and used for experimental purposes in psychophysiology and psychology.

    You can find more information in module documentation:
    https://github.com/c-labpl/qrs_detector

    If you use these modules in a research project, please consider citing it:
    https://zenodo.org/record/583770

    If you use these modules in any other project, please refer to MIT open-source license.


    MIT License

    Copyright (c) 2017 Michał Sznajder, Marta Łukowska

    Permission is hereby granted, free of charge, to any person obtaining a copy
    of this software and associated documentation files (the "Software"), to deal
    in the Software without restriction, including without limitation the rights
    to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
    copies of the Software, and to permit persons to whom the Software is
    furnished to do so, subject to the following conditions:

    The above copyright notice and this permission notice shall be included in all
    copies or substantial portions of the Software.

    THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
    IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
    FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
    AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
    LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
    OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
    SOFTWARE.
    """

    def __init__(self, ecg_data, frequency, verbose=True, log_data=False, plot_data=False, show_plot=False):
        """
        QRSDetectorOffline class initialisation method.
        :param string ecg_data: raw ECG dataset
        :param bool verbose: flag for printing the results
        :param bool log_data: flag for logging the results
        :param bool plot_data: flag for plotting the results to a file
        :param bool show_plot: flag for showing generated results plot - will not show anything if plot is not generated
        """
        # Configuration parameters.
        # Set ECG device frequency in samples per second here.
        self.signal_frequency = frequency
        frequency_scale = frequency/250.0

        self.filter_lowcut = 0.1
        self.filter_highcut = 15.0
        self.filter_order = 1

        # Change proportionally when adjusting frequency (in samples).
        self.integration_window = int(15*frequency_scale)

        #self.findpeaks_limit = 0.01
        # Change proportionally when adjusting frequency (in samples).
        self.findpeaks_spacing = int(50*frequency_scale)

        # Change proportionally when adjusting frequency (in samples).
        self.refractory_period = int(50*frequency_scale)
        self.qrs_peak_filtering_factor = 0.125
        self.noise_peak_filtering_factor = 0.125
        self.qrs_noise_diff_weight = 0.25

        # Loaded ECG data.
        self.ecg_data_raw = ecg_data

        # Measured and calculated values.
        self.filtered_ecg_measurements = None
        self.differentiated_ecg_measurements = None
        self.squared_ecg_measurements = None
        self.integrated_ecg_measurements = None
        self.detected_peaks_indices = None
        self.detected_peaks_values = None

        self.qrs_peak_value = 0.0
        self.noise_peak_value = 0.0
        self.threshold_value = 0.0

        # Detection results.
        self.qrs_peaks_indices = np.array([], dtype=int)
        self.noise_peaks_indices = np.array([], dtype=int)

        # Final ECG data and QRS detection results array - samples with detected QRS are marked with 1 value.
        self.ecg_data_detected = None

        # Run whole detector flow.
        self.detect_peaks()
        self.detect_qrs()

        if verbose:
            self.print_detection_data()

        if log_data:
            self.log_path = "{:s}QRS_offline_detector_log_{:s}.csv".format(LOG_DIR,
                                                                           strftime("%Y_%m_%d_%H_%M_%S", gmtime()))
            self.log_detection_data()

        if plot_data:
            self.plot_path = "{:s}QRS_offline_detector_plot_{:s}.png".format(PLOT_DIR,
                                                                             strftime("%Y_%m_%d_%H_%M_%S", gmtime()))
            self.plot_detection_data(show_plot=show_plot)

    """ECG measurements data processing methods."""

    def detect_peaks(self):
        """
        Method responsible for extracting peaks from loaded ECG measurements data through measurements processing.
        """
        # Extract measurements from loaded ECG data.
        ecg_measurements = self.ecg_data_raw[:, 1]

        # Measurements filtering - 0-15 Hz band pass filter.
        self.filtered_ecg_measurements = self.bandpass_filter(ecg_measurements, lowcut=self.filter_lowcut,
                                                              highcut=self.filter_highcut, signal_freq=self.signal_frequency,
                                                              filter_order=self.filter_order)
        self.filtered_ecg_measurements[:5] = self.filtered_ecg_measurements[5]

        # Derivative - provides QRS slope information.
        self.differentiated_ecg_measurements = np.ediff1d(
            self.filtered_ecg_measurements)

        # Squaring - intensifies values received in derivative.
        self.squared_ecg_measurements = self.differentiated_ecg_measurements ** 2

        # Moving-window integration.
        self.integrated_ecg_measurements = np.convolve(
            self.squared_ecg_measurements, np.ones(self.integration_window))

        # Fiducial mark - peak detection on integrated measurements.
        self.detected_peaks_indices = self.findpeaks(data=self.integrated_ecg_measurements,
                                                     limit=None,
                                                     spacing=self.findpeaks_spacing)

        self.detected_peaks_values = self.integrated_ecg_measurements[self.detected_peaks_indices]

    """QRS detection methods."""

    def detect_qrs(self):
        """
        Method responsible for classifying detected ECG measurements peaks either as noise or as QRS complex (heart beat).
        """
        for detected_peak_index, detected_peaks_value in zip(self.detected_peaks_indices, self.detected_peaks_values):

            try:
                last_qrs_index = self.qrs_peaks_indices[-1]
            except IndexError:
                last_qrs_index = 0

            # After a valid QRS complex detection, there is a 200 ms refractory period before next one can be detected.
            if detected_peak_index - last_qrs_index > self.refractory_period or not self.qrs_peaks_indices.size:
                # Peak must be classified either as a noise peak or a QRS peak.
                # To be classified as a QRS peak it must exceed dynamically set threshold value.
                if detected_peaks_value > self.threshold_value:
                    self.qrs_peaks_indices = np.append(
                        self.qrs_peaks_indices, detected_peak_index)

                    # Adjust QRS peak value used later for setting QRS-noise threshold.
                    self.qrs_peak_value = self.qrs_peak_filtering_factor * detected_peaks_value + \
                        (1 - self.qrs_peak_filtering_factor) * self.qrs_peak_value
                else:
                    self.noise_peaks_indices = np.append(
                        self.noise_peaks_indices, detected_peak_index)

                    # Adjust noise peak value used later for setting QRS-noise threshold.
                    self.noise_peak_value = self.noise_peak_filtering_factor * detected_peaks_value + \
                        (1 - self.noise_peak_filtering_factor) * \
                        self.noise_peak_value

                # Adjust QRS-noise threshold value based on previously detected QRS or noise peaks value.
                self.threshold_value = self.noise_peak_value + \
                    self.qrs_noise_diff_weight * \
                    (self.qrs_peak_value - self.noise_peak_value)

        # Create array containing both input ECG measurements data and QRS detection indication column.
        # We mark QRS detection with '1' flag in 'qrs_detected' log column ('0' otherwise).
        measurement_qrs_detection_flag = np.zeros(
            [len(self.ecg_data_raw[:, 1]), 1])
        measurement_qrs_detection_flag[self.qrs_peaks_indices] = 1
        self.ecg_data_detected = np.append(
            self.ecg_data_raw, measurement_qrs_detection_flag, 1)

    """Results reporting methods."""

    def print_detection_data(self):
        """
        Method responsible for printing the results.
        """
        print("qrs peaks indices")
        print(self.qrs_peaks_indices)
        print("noise peaks indices")
        print(self.noise_peaks_indices)

    def log_detection_data(self):
        """
        Method responsible for logging measured ECG and detection results to a file.
        """
        with open(self.log_path, "wb") as fin:
            fin.write(b"timestamp,ecg_measurement,qrs_detected\n")
            np.savetxt(fin, self.ecg_data_detected, delimiter=",")

    def plot_detection_data(self, show_plot=False):
        """
        Method responsible for plotting detection results.
        :param bool show_plot: flag for plotting the results and showing plot
        """
        def plot_data(axis, data, title='', fontsize=10):
            axis.set_title(title, fontsize=fontsize)
            axis.grid(which='both', axis='both', linestyle='--')
            axis.plot(data, color="salmon", zorder=1)

        def plot_points(axis, values, indices):
            axis.scatter(
                x=indices, y=values[indices], c="black", s=50, zorder=2)

        plt.close('all')
        fig, axarr = plt.subplots(6, sharex=True, figsize=(15, 18))

        plot_data(axis=axarr[0], data=self.ecg_data_raw[:,
                                                        1], title='Raw ECG measurements')
        plot_data(axis=axarr[1], data=self.filtered_ecg_measurements,
                  title='Filtered ECG measurements')
        plot_data(axis=axarr[2], data=self.differentiated_ecg_measurements,
                  title='Differentiated ECG measurements')
        plot_data(axis=axarr[3], data=self.squared_ecg_measurements,
                  title='Squared ECG measurements')
        plot_data(axis=axarr[4], data=self.integrated_ecg_measurements,
                  title='Integrated ECG measurements with QRS peaks marked (black)')
        plot_points(
            axis=axarr[4], values=self.integrated_ecg_measurements, indices=self.qrs_peaks_indices)
        plot_data(axis=axarr[5], data=self.ecg_data_detected[:, 1],
                  title='Raw ECG measurements with QRS peaks marked (black)')
        plot_points(
            axis=axarr[5], values=self.ecg_data_detected[:, 1], indices=self.qrs_peaks_indices)

        plt.tight_layout()
        fig.savefig(self.plot_path)

        if show_plot:
            plt.show()

        plt.close()

    """Tools methods."""

    def bandpass_filter(self, data, lowcut, highcut, signal_freq, filter_order):
        """
        Method responsible for creating and applying Butterworth filter.
        :param deque data: raw data
        :param float lowcut: filter lowcut frequency value
        :param float highcut: filter highcut frequency value
        :param int signal_freq: signal frequency in samples per second (Hz)
        :param int filter_order: filter order
        :return array: filtered data
        """
        nyquist_freq = 0.5 * signal_freq
        low = lowcut / nyquist_freq
        high = highcut / nyquist_freq
        b, a = butter(filter_order, [low, high], btype="band")
        y = lfilter(b, a, data)
        return y

    def findpeaks(self, data, spacing=1, limit=None):
        """
        Janko Slavic peak detection algorithm and implementation.
        https://github.com/jankoslavic/py-tools/tree/master/findpeaks
        Finds peaks in `data` which are of `spacing` width and >=`limit`.
        :param ndarray data: data
        :param float spacing: minimum spacing to the next peak (should be 1 or more)
        :param float limit: peaks should have value greater or equal
        :return array: detected peaks indexes array
        """
        len = data.size
        x = np.zeros(len + 2 * spacing)
        x[:spacing] = data[0] - 1.e-6
        x[-spacing:] = data[-1] - 1.e-6
        x[spacing:spacing + len] = data
        peak_candidate = np.zeros(len)
        peak_candidate[:] = True
        for s in range(spacing):
            start = spacing - s - 1
            h_b = x[start: start + len]  # before
            start = spacing
            h_c = x[start: start + len]  # central
            start = spacing + s + 1
            h_a = x[start: start + len]  # after
            peak_candidate = np.logical_and(
                peak_candidate, np.logical_and(h_c >= h_b, h_c > h_a))

        ind = np.argwhere(peak_candidate)
        ind = ind.reshape(ind.size)
        if limit is not None:
            ind = ind[data[ind] > limit]
        else:
            limit = np.mean(data[ind])/2
            ind = ind[data[ind] > limit]

        return ind


if __name__ == "__main__":
    ecg_data, _ = wfdb.rdsamp('data/CPSC/A0002')
    qrs_detector = QRSDetectorOffline(ecg_data, frequency=500, verbose=False)
    rpeaks = qrs_detector.qrs_peaks_indices
    print(rpeaks)
    print(tools.get_heart_rate(rpeaks, sampling_rate=500))

    

In [ ]:

'''

# QRS Complex Detector

모듈은 ECG 신호 기반에서 QRS 복합 검출의 오프라인 Python 구현입니다
Pan-Tomkins 알고리즘: Pan J, Tompkins W.J., 실시간 QRS 탐지 알고리즘,
생물 의학 공학에 관한 IEEE 거래, Vol. BME-32, No. 3, 1985년 3월, 230-236 페이지.

QRS 복합체는 인간 심장의 오른쪽 및 왼쪽 심실의 탈분극에 해당합니다. 이것은 심전도 신호 중에서 가장 시각적으로 분명한 부분입니다. QRS 복합 검출은 시간 영역 ECG 신호 분석, 즉 심박수 변동성에 필수적입니다. 두 개의 연속적인 R 피크 사이의 시간에 해당하는 비트 간 간격(RR 간격) 값을 계산할 수 있습니다. 따라서 QRS 복합 검출기는 심전도 기반 심장 수축 검출기입니다.

오프라인 버전은 사전 기록된 심전도 신호 데이터 세트(예: .csv 형식으로 저장)에서 QRS 복합을 감지합니다.






제공된 코드 조각은 'QRSDetectorOffline'라는 클래스를 정의합니다. 이 클래스는 심전도(ECG) 신호의 오프라인 QRS 감지에 사용됩니다. 코드의 내용을 설명해드리겠습니다.

import 문은 코드 실행에 필요한 라이브러리와 모듈을 가져옵니다.

'LOG_DIR' 및 'PLOT_DIR' 변수는 로그 파일과 플롯 파일을 저장할 디렉토리 경로를 지정합니다.

'QRSDetectorOffline' 클래스가 정의됩니다. 이 클래스에는 제공된 ECG 데이터, 주파수 및 자세한 정보 출력 여부, 로깅 여부, 플롯 작성 여부, 플롯 표시 여부 등을 나타내는 여러 플래그를 초기화하는 'init' 메서드가 있습니다.

클래스에는 ECG 장치 주파수, 필터 설정, 통합 창, 피크 감지 간격, 회복 기간 등 여러 구성 매개변수가 있습니다.

클래스는 로드된 ECG 데이터, 필터링된 측정값, 미분된 측정값, 제곱된 측정값, 통합된 측정값, 감지된 피크 인덱스 및 감지된 피크 값 등을 저장하기 위한 속성을 초기화합니다.

클래스는 또한 QRS 피크 값, 노이즈 피크 값, 임계값, QRS 피크 인덱스, 노이즈 피크 인덱스 및 감지된 QRS가 표시된 최종 ECG 데이터를 저장하기 위한 속성을 초기화합니다.

클래스에는 피크를 감지하고 ECG 측정값 데이터를 처리하는 메서드가 있습니다. 'detect_peaks' 메서드는 측정값 필터링을 수행합니다.








'''


In [ ]:
'''

이 코드는 QRS 감지를 수행하는 메서드들을 포함한 'QRSDetectorOffline' 클래스의 정의를 보여줍니다. 코드에서 각 메서드의 역할은 다음과 같습니다:

detect_peaks: 로드된 ECG 측정값 데이터에서 피크를 추출하는 메서드입니다. 측정값 데이터를 추출하고, 필터링을 수행한 후, 미분, 제곱, 이동 윈도우 통합을 거친 뒤, 감지된 피크 인덱스와 값을 저장합니다.

detect_qrs: 감지된 ECG 측정값 피크를 노이즈 또는 QRS 복합체(심박동)로 분류하는 메서드입니다. 
이 메서드는 QRS 피크가 이전에 검출된 QRS와 일정 시간(200ms) 이상의 간격을 두고 등장해야 한다는 규칙을 적용합니다.
 또한, 동적으로 설정된 임계값을 초과해야만 QRS 피크로 분류합니다. 분류된 QRS 피크와 노이즈 피크의 인덱스와 값을 저장합니다.

print_detection_data: 결과를 출력하는 메서드로, 검출된 QRS 피크 및 노이즈 피크의 인덱스를 출력합니다.

log_detection_data: 측정된 ECG 및 검출 결과를 파일에 기록하는 메서드입니다. 로그 파일에는 
타임스탬프, ECG 측정값, QRS 감지 여부가 기록됩니다.

plot_detection_data: 검출 결과를 시각화하는 메서드로, 
여러 개의 서브플롯에 원본 ECG 측정값부터 감지된 QRS 피크까지의 처리 과정을 그래프로 표현합니다.

bandpass_filter: Butterworth 필터를 생성하고 적용하는 메서드입니다. 주어진 주파수 범위 내의 데이터만을 통과시킵니다.

findpeaks: 피크 감지 알고리즘을 구현한 메서드입니다. 
주어진 데이터에서 특정 간격과 임계값 이상의 피크를 찾아 인덱스 배열로 반환합니다.

코드의 맨 아래에는

 주어진 ECG 데이터를 로드하고
 QRSDetectorOffline 클래스를 인스턴스화한 후, QRS 피크의 인덱스를 출력하고,
해당 인덱스를 사용하여 심박수를 계산하는 예시 코드가 있습니다.


'''

